https://docs.google.com/spreadsheets/d/133q9H0chayhdxMEdC1rdlynld89cbINKHNO-qEfA0RI/edit?gid=38692135#gid=38692135

In [ ]:
!pip install pandas bibtexparser scholarly requests python-docx fuzzywuzzy pybliometrics groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 23.1 MB/s eta 0:00:00
  Created wheel for bibtexparser: filename=bibtexparser-1.4.3-py3-none-any.whl size=43549 sha256=32e2f66a4d37ad64253ba928baea0e9a323170dbac87cc4974814c0ec0e72663
  Stored in directory: /root/.cache/pip/

## Complete Project

In [ ]:
!pip install pandas openpyxl duckduckgo-search requests xlwt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.7 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import bibtexparser
from docx import Document
from fuzzywuzzy import fuzz
import time
import gradio as gr
import matplotlib.pyplot as plt
import json
from groq import Groq
from datetime import datetime

# Search researchers by name
def search_researcher_by_name(name: str, limit: int = 5):
    """
    Search researcher details using Semantic Scholar API by name.
    Returns list of author IDs and details.
    """
    url = "https://api.semanticscholar.org/graph/v1/author/search"
    params = {
        "query": name,
        "limit": limit,
        "fields": "name,affiliations,paperCount,hIndex,url"
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        authors = data.get("data", [])
        results = [
            {
                "authorId": author.get("authorId"),
                "name": author.get("name"),
                "affiliations": author.get("affiliations", []),
                "paperCount": author.get("paperCount"),
                "hIndex": author.get("hIndex"),
                "url": author.get("url"),
            }
            for author in authors
        ]
        return results, None
    except requests.exceptions.RequestException as e:
        return [], f"Error fetching authors: {str(e)}"

# Safe API request
def safe_request(url, headers, params):
    """Safe API call"""
    try:
        r = requests.get(url, headers=headers, params=params, timeout=10)
        r.raise_for_status()
        return r.json(), None
    except requests.exceptions.RequestException as e:
        return {}, f"Error: {str(e)}"

# Get author papers by ID
def get_author_papers(author_id, author_name, api_key=None, limit=50, export_csv=False, export_excel=False):
    headers = {"x-api-key": api_key} if api_key else {}

    papers_url = f"https://api.semanticscholar.org/graph/v1/author/{author_id}/papers"
    params = {"limit": limit, "fields": "title,year,venue,citationCount,abstract,fieldsOfStudy,openAccessPdf,authors"}
    papers, error = safe_request(papers_url, headers, params)

    if error:
        return {"error": error}, None

    papers = papers.get("data", [])

    papers_list = [
        {
            "title": paper.get("title"),
            "year": paper.get("year"),
            "venue": paper.get("venue"),
            "citations": paper.get("citationCount"),
            "authors": author_name,
            "abstract": paper.get("abstract"),
            "fields_of_study": ",".join(paper.get("fieldsOfStudy", []) or []),
            "open_access_pdf": paper.get("openAccessPdf", {}).get("url", None),
            "authors_api": ",".join([a.get("name", "") for a in paper.get("authors", [])])
        }
        for paper in papers
    ]

    papers_df = pd.DataFrame(papers_list)
    papers_df = papers_df.sort_values(by="year", ascending=True).reset_index(drop=True)

    if export_csv:
        papers_df.to_csv(f"{author_name}_papers.csv", index=False)
    if export_excel:
        papers_df.to_excel(f"{author_name}_papers.xlsx", index=False)

    return {"authorId": author_id, "name": author_name}, papers_df

# Parse Excel for author names
def parse_excel_for_authors(file_path):
    try:
        df = pd.read_excel(file_path, engine='xlrd' if file_path.endswith('.xls') else 'openpyxl')
        df = df.rename(columns=lambda x: x.strip().lower())
        authors_col = next((col for col in df.columns if "author" in col.lower()), None)
        if not authors_col:
            return [], "Excel file must contain a column with 'author' in its name"
        authors = df[authors_col].dropna().unique().tolist()
        if not authors:
            return [], "No valid author names found in Excel file"
        return authors, None
    except Exception as e:
        return [], f"Error parsing Excel file: {str(e)}"

# Parse Excel for publication data
def parse_excel(file_path):
    try:
        df = pd.read_excel(file_path, engine='xlrd' if file_path.endswith('.xls') else 'openpyxl')
        df = df.rename(columns=lambda x: x.strip().lower())
        title_col = next((col for col in df.columns if "title" in col.lower()), None)
        authors_col = next((col for col in df.columns if "author" in col.lower()), None)
        year_col = next((col for col in df.columns if "year" in col.lower()), None)
        if not all([title_col, authors_col, year_col]):
            raise ValueError("Excel file must contain columns for title, authors, and year")
        df_clean = df[[title_col, authors_col, year_col]].copy()
        df_clean.columns = ['title', 'authors', 'year']
        return df_clean, None
    except Exception as e:
        return pd.DataFrame(), f"Error parsing Excel file: {str(e)}"

# Parse bibtex
def parse_bibtex(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as bib_file:
            bib_database = bibtexparser.load(bib_file)
        records = [
            {
                'title': entry.get('title', '').replace('{', '').replace('}', ''),
                'authors': entry.get('author', '').replace('\n', ' '),
                'year': entry.get('year', '')
            }
            for entry in bib_database.entries
        ]
        return pd.DataFrame(records), None
    except Exception as e:
        return pd.DataFrame(), f"Error parsing BibTeX file: {str(e)}"

# Enrich with Semantic Scholar
def enrich_with_semantic_scholar(df, api_key=None, progress=gr.Progress()):
    base_url = "https://api.semanticscholar.org/graph/v1/paper/search/match"
    headers = {"x-api-key": api_key} if api_key else {}
    df['citations'] = None
    df['venue'] = None
    df['abstract'] = None
    df['fields_of_study'] = None
    df['open_access_pdf'] = None
    df['authors_api'] = None

    client = Groq(api_key="Add your key Here")

    for i, (idx, row) in enumerate(df.iterrows()):
        title = row['title']
        year = row['year']
        params = {
            "query": title,
            "fields": "title,venue,citationCount,abstract,fieldsOfStudy,openAccessPdf,authors",
            "year": str(year) if pd.notna(year) else None
        }
        paper_found = False
        for attempt in range(7):
            try:
                response = requests.get(base_url, params=params, headers=headers, timeout=10)
                response.raise_for_status()
                data = response.json()
                if "data" in data and data["data"]:
                    paper = data["data"][0]
                    if fuzz.ratio(title.lower(), paper.get("title", "").lower()) > 85:
                        df.at[idx, 'citations'] = paper.get("citationCount", None)
                        df.at[idx, 'venue'] = paper.get("venue", None)
                        df.at[idx, 'abstract'] = paper.get("abstract", None)
                        df.at[idx, 'fields_of_study'] = ",".join(paper.get("fieldsOfStudy", []) or [])
                        df.at[idx, 'open_access_pdf'] = paper.get("openAccessPdf", {}).get("url", None)
                        df.at[idx, 'authors_api'] = ",".join([a.get("name", "") for a in paper.get("authors", [])])
                        paper_found = True
                break
            except requests.exceptions.HTTPError as e:
                if response.status_code == 429:
                    time.sleep(2**attempt)
                    continue
                break
            except Exception:
                break

        if any(df.at[idx, col] is None for col in ['citations', 'venue', 'abstract', 'fields_of_study', 'open_access_pdf', 'authors_api']):
            year_str = f" published around {year}" if pd.notna(year) else ""
            prompt = f"""
            Provide details for the research paper titled "{title}"{year_str} including citations, venue, abstract, fields of study, open access PDF, and authors.
            Return the response strictly in the following JSON format without any additional text:
            {{
                "citations": <integer citation count or null>,
                "venue": "<string venue or journal name or null>",
                "abstract": "<string abstract or null>",
                "fields_of_study": [<array of strings for fields of study or empty array>],
                "open_access_pdf": "<string URL to open access PDF or null>",
                "authors_api": [<array of author names as strings or empty array>]
            }}
            If information is not available, use null or empty array as appropriate.
            """
            try:
                completion = client.chat.completions.create(
                    model="llama3-70b-8192",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.2,
                    max_tokens=1024
                )
                response_text = completion.choices[0].message.content.strip()
                data = json.loads(response_text)
                if df.at[idx, 'citations'] is None:
                    df.at[idx, 'citations'] = data.get('citations')
                if df.at[idx, 'venue'] is None:
                    df.at[idx, 'venue'] = data.get('venue')
                if df.at[idx, 'abstract'] is None:
                    df.at[idx, 'abstract'] = data.get('abstract')
                if df.at[idx, 'fields_of_study'] is None:
                    df.at[idx, 'fields_of_study'] = ",".join(data.get('fields_of_study', [])) if data.get('fields_of_study') else None
                if df.at[idx, 'open_access_pdf'] is None:
                    df.at[idx, 'open_access_pdf'] = data.get('open_access_pdf')
                if df.at[idx, 'authors_api'] is None:
                    df.at[idx, 'authors_api'] = ",".join(data.get('authors_api', [])) if data.get('authors_api') else None
            except Exception:
                pass
        progress((i + 1) / len(df))
    return df

# Filter by year
def filter_by_year(df, start_year, end_year):
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    return df[(df['year'] >= start_year) & (df['year'] <= end_year) & df['year'].notna()]

# Generate year summary
def generate_year_summary(df):
    df['citations'] = pd.to_numeric(df['citations'], errors='coerce').fillna(0)
    summary = df.groupby('year').agg({
        'title': list,
        'year': 'count',
        'citations': 'sum'
    }).rename(columns={'year': 'publication_count', 'citations': 'total_citations'}).reset_index()
    return summary

# Export summary to Excel
def export_summary_to_excel(df, summary, author_name, file_path="summary.xlsx"):
    with pd.ExcelWriter(file_path) as writer:
        df.to_excel(writer, sheet_name="Filtered_Publications", index=False)
        summary.to_excel(writer, sheet_name="Year_Summary", index=False)

# Export summary to Word
def export_summary_to_word(summary, author_name, file_path="summary.docx"):
    doc = Document()
    doc.add_heading(f'Year-wise Publication Summary for {author_name}', 0)
    for _, row in summary.iterrows():
        doc.add_heading(f"Year: {row['year']} ({row['publication_count']} publications)", level=1)
        for title in row['title']:
            doc.add_paragraph(f"• {title}", style='List Bullet')
    doc.save(file_path)

# Generate plots
def generate_plots(summary):
    fig1, ax1 = plt.subplots(figsize=(10, 6))
    ax1.plot(summary['year'], summary['publication_count'], marker='o', color='#00ffcc')
    ax1.set_title("Publication Count per Year", color='white')
    ax1.set_xlabel("Year", color='white')
    ax1.set_ylabel("Publication Count", color='white')
    ax1.grid(True, color=(1, 1, 1, 0.2))
    ax1.set_facecolor('#1a1a2e')
    fig1.set_facecolor('#1a1a2e')
    ax1.tick_params(colors='white')

    fig2, ax2 = plt.subplots(figsize=(10, 6))
    ax2.plot(summary['year'], summary['total_citations'], marker='o', color='#ff007a')
    ax2.set_title("Total Citations per Year", color='white')
    ax2.set_xlabel("Year", color='white')
    ax2.set_ylabel("Total Citations", color='white')
    ax2.grid(True, color=(1, 1, 1, 0.2))
    ax2.set_facecolor('#1a1a2e')
    fig2.set_facecolor('#1a1a2e')
    ax2.tick_params(colors='white')

    return fig1, fig2

# Handle author selection for Researcher Name input
def get_author_selection(author_list, error_message=None):
    if error_message:
        return None, None, None, None, None, None, None, f"<p style='color:#ff4d4d; font-weight:bold;'>Error: {error_message}</p>", [], gr.update(visible=False)

    if not author_list:
        return None, None, None, None, None, None, None, "<p style='color:#ff4d4d; font-weight:bold;'>No authors found. Please check the input and try again.</p>", [], gr.update(visible=False)

    authors_df = pd.DataFrame(author_list)
    authors_df = authors_df[['authorId', 'name', 'affiliations', 'paperCount', 'hIndex', 'url']]

    return (
        None, None, None, None, None, None, authors_df,
        "<p style='color:#00ffcc; font-weight:bold;'>Please enter an Author ID from the table below</p>",
        author_list, gr.update(visible=True, placeholder="Enter Author ID (e.g., 123456)")
    )

# Process function
def process_function(input_type, excel_file, author_name, start_year, end_year, api_key, selected_author_id=None, author_list=None, progress=gr.Progress()):
    try:
        if input_type == "Excel File":
            if excel_file is None:
                raise ValueError("Please upload a valid Excel file (.xls or .xlsx)")
            author_names, error = parse_excel_for_authors(excel_file)
            if error:
                return None, None, None, None, None, None, None, f"<p style='color:#ff4d4d; font-weight:bold;'>{error}</p>", [], gr.update(visible=False)
            if not author_names:
                raise ValueError("No valid author names found in Excel file")

            # Fetch papers for all authors in the Excel file
            all_papers = []
            author_info = {"name": "Multiple Authors", "authorId": None}
            for name in author_names:
                authors, error = search_researcher_by_name(name, limit=1)  # Take the first match for simplicity
                if error:
                    return None, None, None, None, None, None, None, f"<p style='color:#ff4d4d; font-weight:bold;'>{error}</p>", [], gr.update(visible=False)
                if authors:
                    author = authors[0]
                    _, papers_df = get_author_papers(author['authorId'], author['name'], api_key=api_key, limit=50)
                    if papers_df is not None and not papers_df.empty:
                        all_papers.append(papers_df)

            # Combine papers from all authors
            if all_papers:
                df = pd.concat(all_papers).drop_duplicates(subset=['title']).reset_index(drop=True)
            else:
                df = pd.DataFrame()

            # Parse Excel for additional papers
            df_excel, error = parse_excel(excel_file)
            if error:
                return None, None, None, None, None, None, None, f"<p style='color:#ff4d4d; font-weight:bold;'>{error}</p>", [], gr.update(visible=False)
            df_excel = enrich_with_semantic_scholar(df_excel, api_key, progress)
            df = pd.concat([df, df_excel]).drop_duplicates(subset=['title']).reset_index(drop=True)

        elif input_type == "Researcher Name":
            if not author_name:
                raise ValueError("Please provide a researcher name")

            if not selected_author_id:
                authors, error = search_researcher_by_name(author_name, limit=5)
                if error:
                    return get_author_selection([], error)
                return get_author_selection(authors)

            if not author_list:
                authors, error = search_researcher_by_name(author_name, limit=5)
                if error:
                    return get_author_selection([], error)
                author_list = authors

            selected_author = next((a for a in author_list if a['authorId'] == selected_author_id), None)
            if not selected_author:
                return get_author_selection(author_list, "Invalid author ID selected. Please choose an ID from the table.")

            author_info, df = get_author_papers(selected_author['authorId'], selected_author['name'], api_key=api_key, limit=50)
            if df is None or df.empty:
                raise ValueError(author_info.get("error", "No papers found for the selected author"))

            df = enrich_with_semantic_scholar(df, api_key, progress)

        else:
            raise ValueError("Invalid input type")

        filtered_df = filter_by_year(df, start_year, end_year)
        summary = generate_year_summary(filtered_df)
        plot1, plot2 = generate_plots(summary)
        timestamp = int(time.time())
        excel_path = f"summary_{author_info['name']}_{timestamp}.xlsx"
        word_path = f"summary_{author_info['name']}_{timestamp}.docx"
        export_summary_to_excel(filtered_df, summary, author_info['name'], excel_path)
        export_summary_to_word(summary, author_info['name'], word_path)
        return (
            filtered_df, summary, plot1, plot2, excel_path, word_path, None,
            f"<p style='color:#00ffcc; font-weight:bold;'>Processing complete for {author_info['name']}</p>",
            [], gr.update(visible=False)
        )
    except Exception as e:
        return None, None, None, None, None, None, None, f"<p style='color:#ff4d4d; font-weight:bold;'>Error: {str(e)}</p>", author_list or [], gr.update(visible=False)

# Web3-inspired CSS
css = """
body {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    color: #e0e0e0;
}
h1 {
    color: #00ffcc;
    text-align: center;
    margin-bottom: 20px;
    text-shadow: 0 0 10px rgba(0, 255, 204, 0.5);
}
button {
    background: linear-gradient(45deg, #ff007a, #00ffcc);
    color: white;
    border: none;
    padding: 12px 24px;
    font-size: 16px;
    border-radius: 8px;
    cursor: pointer;
    transition: transform 0.2s, box-shadow 0.2s;
}
button:hover {
    transform: translateY(-2px);
    box-shadow: 0 0 15px rgba(0, 255, 204, 0.5);
}
button:active {
    transform: translateY(0);
}
.label {
    font-weight: 600;
    color: #00ffcc;
    margin-bottom: 8px;
}
.gr-column, .gr-row {
    background: rgba(255, 255, 255, 0.05);
    backdrop-filter: blur(10px);
    border-radius: 12px;
    padding: 20px;
    margin: 10px;
    border: 1px solid rgba(255, 255, 255, 0.1);
}
input, .gr-textbox, .gr-file, .gr-slider, .gr-radio {
    background: rgba(255, 255, 255, 0.1);
    border: 1px solid rgba(255, 255, 255, 0.2);
    color: #e0e0e0;
    border-radius: 8px;
    padding: 10px;
}
input:focus, .gr-textbox:focus {
    border-color: #00ffcc;
    box-shadow: 0 0 8px rgba(0, 255, 204, 0.3);
}
.gr-dataframe {
    background: rgba(255, 255, 255, 0.05);
    border-radius: 8px;
    color: #e0e0e0;
}
.gr-dataframe table {
    border-collapse: collapse;
}
.gr-dataframe th, .gr-dataframe td {
    border: 1px solid rgba(255, 255, 255, 0.1);
    padding: 8px;
}
.gr-dataframe th {
    background: rgba(0, 255, 204, 0.1);
    color: #00ffcc;
}
.gr-html {
    font-size: 16px;
}
"""

with gr.Blocks(css=css) as demo:
    gr.HTML("<h1>🌌 Publication Analyzer</h1>")
    gr.Markdown("""
    **Welcome to the Future of Research Analysis**
    Upload an Excel file (.xls or .xlsx) with publication data (must include 'title', 'authors', and 'year' columns) or select 'Researcher Name' to enter a name.
    For Excel files, papers are fetched directly. For researcher names, select an author ID from the results.
    Set a year range, add a Semantic Scholar API key (optional), and hit Submit to explore papers, summaries, and visualizations!
    """, elem_classes="markdown")

    with gr.Row():
        with gr.Column():
            input_type = gr.Radio(
                ["Excel File", "Researcher Name"], label="Input Type", value="Excel File",
                elem_classes="radio"
            )
            file_input = gr.File(label="Upload Excel File (.xls or .xlsx)", file_types=[".xls", ".xlsx"], visible=True)
            author_input = gr.Textbox(
                label="Researcher Name", visible=False, placeholder="Enter full name, e.g., John Doe"
            )
            api_key_input = gr.Textbox(
                label="Semantic Scholar API Key (optional)", type="password",
                placeholder="Enter your API key"
            )
            current_year = datetime.now().year
            start_year_input = gr.Slider(1900, current_year, value=1900, step=1, label="Start Year")
            end_year_input = gr.Slider(1900, current_year, value=current_year, step=1, label="End Year")
            author_id_input = gr.Textbox(
                label="Selected Author ID", visible=False, placeholder="Enter Author ID (e.g., 123456)"
            )
            submit_button = gr.Button("Submit 🚀", variant="primary")

    with gr.Row():
        with gr.Column():
            authors_df_output = gr.DataFrame(label="Available Authors")
            filtered_df_output = gr.DataFrame(label="Filtered Publications")
            summary_output = gr.DataFrame(label="Year Summary")
        with gr.Column():
            plot1_output = gr.Plot(label="Publication Count")
            plot2_output = gr.Plot(label="Total Citations")

    with gr.Row():
        excel_download = gr.File(label="Download Excel")
        word_download = gr.File(label="Download Word")
        status_output = gr.HTML(label="Status")

    author_list_state = gr.State(value=[])

    def update_inputs(input_type):
        return (
            gr.update(visible=(input_type == "Excel File")),
            gr.update(visible=(input_type == "Researcher Name")),
            gr.update(visible=False)
        )

    input_type.change(
        fn=update_inputs,
        inputs=input_type,
        outputs=[file_input, author_input, author_id_input]
    )

    def process_wrapper(input_type, file, author_name, api_key, start_year, end_year, selected_author_id, author_list):
        excel_file = file.name if file else None
        filtered_df, summary, plot1, plot2, excel_path, word_path, authors_df, status, new_author_list, author_id_visible = process_function(
            input_type, excel_file, author_name, start_year, end_year, api_key, selected_author_id, author_list
        )
        return (
            filtered_df, summary, plot1, plot2, excel_path, word_path, authors_df, status, new_author_list, author_id_visible
        )

    submit_button.click(
        process_wrapper,
        inputs=[input_type, file_input, author_input, api_key_input, start_year_input, end_year_input, author_id_input, author_list_state],
        outputs=[
            filtered_df_output, summary_output, plot1_output, plot2_output, excel_download, word_download,
            authors_df_output, status_output, author_list_state, author_id_input
        ]
    )

if __name__ == "__main__":
    demo.launch()

/usr/local/lib/python3.12/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/tmp/ipython-input-558551881.py:459: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a775eac9ddc952236f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install pandas bibtexparser scholarly requests python-docx fuzzywuzzy pybliometrics groq

"""## Complete Project"""

!pip install pandas openpyxl duckduckgo-search requests xlwt

In [ ]:

import requests
import pandas as pd
import bibtexparser
from docx import Document
from fuzzywuzzy import fuzz
import time
import gradio as gr
import matplotlib.pyplot as plt
import json
from groq import Groq
from datetime import datetime

# Search researchers by name
def search_researcher_by_name(name: str, limit: int = 5):
    """
    Search researcher details using Semantic Scholar API by name.
    Returns list of author IDs and details.
    """
    url = "https://api.semanticscholar.org/graph/v1/author/search"
    params = {
        "query": name,
        "limit": limit,
        "fields": "name,affiliations,paperCount,hIndex,url"
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        authors = data.get("data", [])
        results = [
            {
                "authorId": author.get("authorId"),
                "name": author.get("name"),
                "affiliations": author.get("affiliations", []),
                "paperCount": author.get("paperCount"),
                "hIndex": author.get("hIndex"),
                "url": author.get("url"),
            }
            for author in authors
        ]
        return results, None
    except requests.exceptions.RequestException as e:
        return [], f"Error fetching authors: {str(e)}"

# Safe API request
def safe_request(url, headers, params):
    """Safe API call"""
    try:
        r = requests.get(url, headers=headers, params=params, timeout=10)
        r.raise_for_status()
        return r.json(), None
    except requests.exceptions.RequestException as e:
        return {}, f"Error: {str(e)}"

# Get author papers by ID
def get_author_papers(author_id, author_name, api_key=None, limit=50, export_csv=False, export_excel=False):
    headers = {"x-api-key": api_key} if api_key else {}

    papers_url = f"https://api.semanticscholar.org/graph/v1/author/{author_id}/papers"
    params = {"limit": limit, "fields": "title,year,venue,citationCount,abstract,fieldsOfStudy,openAccessPdf,authors"}
    papers, error = safe_request(papers_url, headers, params)

    if error:
        return {"error": error}, None

    papers = papers.get("data", [])

    papers_list = [
        {
            "title": paper.get("title"),
            "year": paper.get("year"),
            "venue": paper.get("venue"),
            "citations": paper.get("citationCount"),
            "authors": author_name,
            "abstract": paper.get("abstract"),
            "fields_of_study": ",".join(paper.get("fieldsOfStudy", []) or []),
            "open_access_pdf": paper.get("openAccessPdf", {}).get("url", None),
            "authors_api": ",".join([a.get("name", "") for a in paper.get("authors", [])])
        }
        for paper in papers
    ]

    papers_df = pd.DataFrame(papers_list)
    papers_df = papers_df.sort_values(by="year", ascending=True).reset_index(drop=True)

    if export_csv:
        papers_df.to_csv(f"{author_name}_papers.csv", index=False)
    if export_excel:
        papers_df.to_excel(f"{author_name}_papers.xlsx", index=False)

    return {"authorId": author_id, "name": author_name}, papers_df

# Parse Excel for author names
def parse_excel_for_authors(file_path):
    try:
        df = pd.read_excel(file_path, engine='xlrd' if file_path.endswith('.xls') else 'openpyxl')
        df = df.rename(columns=lambda x: x.strip().lower())
        authors_col = next((col for col in df.columns if "author" in col.lower()), None)
        if not authors_col:
            return [], "Excel file must contain a column with 'author' in its name"
        authors = df[authors_col].dropna().unique().tolist()
        if not authors:
            return [], "No valid author names found in Excel file"
        return authors, None
    except Exception as e:
        return [], f"Error parsing Excel file: {str(e)}"

# Parse Excel for publication data
def parse_excel(file_path):
    try:
        df = pd.read_excel(file_path, engine='xlrd' if file_path.endswith('.xls') else 'openpyxl')
        df = df.rename(columns=lambda x: x.strip().lower())
        title_col = next((col for col in df.columns if "title" in col.lower()), None)
        authors_col = next((col for col in df.columns if "author" in col.lower()), None)
        year_col = next((col for col in df.columns if "year" in col.lower()), None)
        if not all([title_col, authors_col, year_col]):
            raise ValueError("Excel file must contain columns for title, authors, and year")
        df_clean = df[[title_col, authors_col, year_col]].copy()
        df_clean.columns = ['title', 'authors', 'year']
        return df_clean, None
    except Exception as e:
        return pd.DataFrame(), f"Error parsing Excel file: {str(e)}"

# Parse bibtex
def parse_bibtex(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as bib_file:
            bib_database = bibtexparser.load(bib_file)
        records = [
            {
                'title': entry.get('title', '').replace('{', '').replace('}', ''),
                'authors': entry.get('author', '').replace('\n', ' '),
                'year': entry.get('year', '')
            }
            for entry in bib_database.entries
        ]
        return pd.DataFrame(records), None
    except Exception as e:
        return pd.DataFrame(), f"Error parsing BibTeX file: {str(e)}"

# Enrich with Semantic Scholar
def enrich_with_semantic_scholar(df, api_key=None, progress=gr.Progress()):
    base_url = "https://api.semanticscholar.org/graph/v1/paper/search/match"
    headers = {"x-api-key": api_key} if api_key else {}
    df['citations'] = None
    df['venue'] = None
    df['abstract'] = None
    df['fields_of_study'] = None
    df['open_access_pdf'] = None
    df['authors_api'] = None

    client = Groq(api_key="Addd your key here")

    for i, (idx, row) in enumerate(df.iterrows()):
        title = row['title']
        year = row['year']
        params = {
            "query": title,
            "fields": "title,venue,citationCount,abstract,fieldsOfStudy,openAccessPdf,authors",
            "year": str(year) if pd.notna(year) else None
        }
        paper_found = False
        for attempt in range(7):
            try:
                response = requests.get(base_url, params=params, headers=headers, timeout=10)
                response.raise_for_status()
                data = response.json()
                if "data" in data and data["data"]:
                    paper = data["data"][0]
                    if fuzz.ratio(title.lower(), paper.get("title", "").lower()) > 85:
                        df.at[idx, 'citations'] = paper.get("citationCount", None)
                        df.at[idx, 'venue'] = paper.get("venue", None)
                        df.at[idx, 'abstract'] = paper.get("abstract", None)
                        df.at[idx, 'fields_of_study'] = ",".join(paper.get("fieldsOfStudy", []) or [])
                        df.at[idx, 'open_access_pdf'] = paper.get("openAccessPdf", {}).get("url", None)
                        df.at[idx, 'authors_api'] = ",".join([a.get("name", "") for a in paper.get("authors", [])])
                        paper_found = True
                break
            except requests.exceptions.HTTPError as e:
                if response.status_code == 429:
                    time.sleep(2**attempt)
                    continue
                break
            except Exception:
                break

        if any(df.at[idx, col] is None for col in ['citations', 'venue', 'abstract', 'fields_of_study', 'open_access_pdf', 'authors_api']):
            year_str = f" published around {year}" if pd.notna(year) else ""
            prompt = f"""
            Provide details for the research paper titled "{title}"{year_str} including citations, venue, abstract, fields of study, open access PDF, and authors.
            Return the response strictly in the following JSON format without any additional text:
            {{
                "citations": <integer citation count or null>,
                "venue": "<string venue or journal name or null>",
                "abstract": "<string abstract or null>",
                "fields_of_study": [<array of strings for fields of study or empty array>],
                "open_access_pdf": "<string URL to open access PDF or null>",
                "authors_api": [<array of author names as strings or empty array>]
            }}
            If information is not available, use null or empty array as appropriate.
            """
            try:
                completion = client.chat.completions.create(
                    model="llama3-70b-8192",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.2,
                    max_tokens=1024
                )
                response_text = completion.choices[0].message.content.strip()
                data = json.loads(response_text)
                if df.at[idx, 'citations'] is None:
                    df.at[idx, 'citations'] = data.get('citations')
                if df.at[idx, 'venue'] is None:
                    df.at[idx, 'venue'] = data.get('venue')
                if df.at[idx, 'abstract'] is None:
                    df.at[idx, 'abstract'] = data.get('abstract')
                if df.at[idx, 'fields_of_study'] is None:
                    df.at[idx, 'fields_of_study'] = ",".join(data.get('fields_of_study', [])) if data.get('fields_of_study') else None
                if df.at[idx, 'open_access_pdf'] is None:
                    df.at[idx, 'open_access_pdf'] = data.get('open_access_pdf')
                if df.at[idx, 'authors_api'] is None:
                    df.at[idx, 'authors_api'] = ",".join(data.get('authors_api', [])) if data.get('authors_api') else None
            except Exception:
                pass
        progress((i + 1) / len(df))
    return df

# Filter by year
def filter_by_year(df, start_year, end_year):
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    return df[(df['year'] >= start_year) & (df['year'] <= end_year) & df['year'].notna()]

# Generate year summary
def generate_year_summary(df):
    df['citations'] = pd.to_numeric(df['citations'], errors='coerce').fillna(0)
    summary = df.groupby('year').agg({
        'title': list,
        'year': 'count',
        'citations': 'sum'
    }).rename(columns={'year': 'publication_count', 'citations': 'total_citations'}).reset_index()
    return summary

# Export summary to Excel
def export_summary_to_excel(df, summary, author_name, file_path="summary.xlsx"):
    with pd.ExcelWriter(file_path) as writer:
        df.to_excel(writer, sheet_name="Filtered_Publications", index=False)
        summary.to_excel(writer, sheet_name="Year_Summary", index=False)

# Export summary to Word
def export_summary_to_word(summary, author_name, file_path="summary.docx"):
    doc = Document()
    doc.add_heading(f'Year-wise Publication Summary for {author_name}', 0)
    for _, row in summary.iterrows():
        doc.add_heading(f"Year: {row['year']} ({row['publication_count']} publications)", level=1)
        for title in row['title']:
            doc.add_paragraph(f"• {title}", style='List Bullet')
    doc.save(file_path)

# Generate plots
def generate_plots(summary):
    fig1, ax1 = plt.subplots(figsize=(10, 6))
    ax1.plot(summary['year'], summary['publication_count'], marker='o', color='#00ffcc')
    ax1.set_title("Publication Count per Year", color='white')
    ax1.set_xlabel("Year", color='white')
    ax1.set_ylabel("Publication Count", color='white')
    ax1.grid(True, color=(1, 1, 1, 0.2))
    ax1.set_facecolor('#1a1a2e')
    fig1.set_facecolor('#1a1a2e')
    ax1.tick_params(colors='white')

    fig2, ax2 = plt.subplots(figsize=(10, 6))
    ax2.plot(summary['year'], summary['total_citations'], marker='o', color='#ff007a')
    ax2.set_title("Total Citations per Year", color='white')
    ax2.set_xlabel("Year", color='white')
    ax2.set_ylabel("Total Citations", color='white')
    ax2.grid(True, color=(1, 1, 1, 0.2))
    ax2.set_facecolor('#1a1a2e')
    fig2.set_facecolor('#1a1a2e')
    ax2.tick_params(colors='white')

    return fig1, fig2

# Handle author selection for Researcher Name input
def get_author_selection(author_list, error_message=None):
    if error_message:
        return None, None, None, None, None, None, None, f"<p style='color:#ff4d4d; font-weight:bold;'>Error: {error_message}</p>", [], gr.update(visible=False)

    if not author_list:
        return None, None, None, None, None, None, None, "<p style='color:#ff4d4d; font-weight:bold;'>No authors found. Please check the input and try again.</p>", [], gr.update(visible=False)

    authors_df = pd.DataFrame(author_list)
    authors_df = authors_df[['authorId', 'name', 'affiliations', 'paperCount', 'hIndex', 'url']]

    return (
        None, None, None, None, None, None, authors_df,
        "<p style='color:#00ffcc; font-weight:bold;'>Please enter an Author ID from the table below</p>",
        author_list, gr.update(visible=True, placeholder="Enter Author ID (e.g., 123456)")
    )

# Process function
def process_function(input_type, excel_file, author_name, start_year, end_year, api_key, selected_author_id=None, author_list=None, progress=gr.Progress()):
    try:
        if input_type == "Excel File":
            if excel_file is None:
                raise ValueError("Please upload a valid Excel file (.xls or .xlsx)")
            author_names, error = parse_excel_for_authors(excel_file)
            if error:
                return None, None, None, None, None, None, None, f"<p style='color:#ff4d4d; font-weight:bold;'>{error}</p>", [], gr.update(visible=False)
            if not author_names:
                raise ValueError("No valid author names found in Excel file")

            # Fetch papers for all authors in the Excel file
            all_papers = []
            author_info = {"name": "Multiple Authors", "authorId": None}
            for name in author_names:
                authors, error = search_researcher_by_name(name, limit=1)  # Take the first match for simplicity
                if error:
                    return None, None, None, None, None, None, None, f"<p style='color:#ff4d4d; font-weight:bold;'>{error}</p>", [], gr.update(visible=False)
                if authors:
                    author = authors[0]
                    _, papers_df = get_author_papers(author['authorId'], author['name'], api_key=api_key, limit=50)
                    if papers_df is not None and not papers_df.empty:
                        all_papers.append(papers_df)

            # Combine papers from all authors
            if all_papers:
                df = pd.concat(all_papers).drop_duplicates(subset=['title']).reset_index(drop=True)
            else:
                df = pd.DataFrame()

            # Parse Excel for additional papers
            df_excel, error = parse_excel(excel_file)
            if error:
                return None, None, None, None, None, None, None, f"<p style='color:#ff4d4d; font-weight:bold;'>{error}</p>", [], gr.update(visible=False)
            df_excel = enrich_with_semantic_scholar(df_excel, api_key, progress)
            df = pd.concat([df, df_excel]).drop_duplicates(subset=['title']).reset_index(drop=True)

        elif input_type == "Researcher Name":
            if not author_name:
                raise ValueError("Please provide a researcher name")

            if not selected_author_id:
                authors, error = search_researcher_by_name(author_name, limit=5)
                if error:
                    return get_author_selection([], error)
                return get_author_selection(authors)

            if not author_list:
                authors, error = search_researcher_by_name(author_name, limit=5)
                if error:
                    return get_author_selection([], error)
                author_list = authors

            selected_author = next((a for a in author_list if a['authorId'] == selected_author_id), None)
            if not selected_author:
                return get_author_selection(author_list, "Invalid author ID selected. Please choose an ID from the table.")

            author_info, df = get_author_papers(selected_author['authorId'], selected_author['name'], api_key=api_key, limit=50)
            if df is None or df.empty:
                raise ValueError(author_info.get("error", "No papers found for the selected author"))

            df = enrich_with_semantic_scholar(df, api_key, progress)

        else:
            raise ValueError("Invalid input type")

        filtered_df = filter_by_year(df, start_year, end_year)
        summary = generate_year_summary(filtered_df)
        plot1, plot2 = generate_plots(summary)
        timestamp = int(time.time())
        excel_path = f"summary_{author_info['name']}_{timestamp}.xlsx"
        word_path = f"summary_{author_info['name']}_{timestamp}.docx"
        export_summary_to_excel(filtered_df, summary, author_info['name'], excel_path)
        export_summary_to_word(summary, author_info['name'], word_path)
        return (
            filtered_df, summary, plot1, plot2, excel_path, word_path, None,
            f"<p style='color:#00ffcc; font-weight:bold;'>Processing complete for {author_info['name']}</p>",
            [], gr.update(visible=False)
        )
    except Exception as e:
        return None, None, None, None, None, None, None, f"<p style='color:#ff4d4d; font-weight:bold;'>Error: {str(e)}</p>", author_list or [], gr.update(visible=False)

# Web3-inspired CSS
css = """
body {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    color: #e0e0e0;
}
h1 {
    color: #00ffcc;
    text-align: center;
    margin-bottom: 20px;
    text-shadow: 0 0 10px rgba(0, 255, 204, 0.5);
}
button {
    background: linear-gradient(45deg, #ff007a, #00ffcc);
    color: white;
    border: none;
    padding: 12px 24px;
    font-size: 16px;
    border-radius: 8px;
    cursor: pointer;
    transition: transform 0.2s, box-shadow 0.2s;
}
button:hover {
    transform: translateY(-2px);
    box-shadow: 0 0 15px rgba(0, 255, 204, 0.5);
}
button:active {
    transform: translateY(0);
}
.label {
    font-weight: 600;
    color: #00ffcc;
    margin-bottom: 8px;
}
.gr-column, .gr-row {
    background: rgba(255, 255, 255, 0.05);
    backdrop-filter: blur(10px);
    border-radius: 12px;
    padding: 20px;
    margin: 10px;
    border: 1px solid rgba(255, 255, 255, 0.1);
}
input, .gr-textbox, .gr-file, .gr-slider, .gr-radio {
    background: rgba(255, 255, 255, 0.1);
    border: 1px solid rgba(255, 255, 255, 0.2);
    color: #e0e0e0;
    border-radius: 8px;
    padding: 10px;
}
input:focus, .gr-textbox:focus {
    border-color: #00ffcc;
    box-shadow: 0 0 8px rgba(0, 255, 204, 0.3);
}
.gr-dataframe {
    background: rgba(255, 255, 255, 0.05);
    border-radius: 8px;
    color: #e0e0e0;
}
.gr-dataframe table {
    border-collapse: collapse;
}
.gr-dataframe th, .gr-dataframe td {
    border: 1px solid rgba(255, 255, 255, 0.1);
    padding: 8px;
}
.gr-dataframe th {
    background: rgba(0, 255, 204, 0.1);
    color: #00ffcc;
}
.gr-html {
    font-size: 16px;
}
"""

with gr.Blocks(css=css) as demo:
    gr.HTML("<h1>🌌 Publication Analyzer</h1>")
    gr.Markdown("""
    **Welcome to the Future of Research Analysis**
    Upload an Excel file (.xls or .xlsx) with publication data (must include 'title', 'authors', and 'year' columns) or select 'Researcher Name' to enter a name.
    For Excel files, papers are fetched directly. For researcher names, select an author ID from the results.
    Set a year range, add a Semantic Scholar API key (optional), and hit Submit to explore papers, summaries, and visualizations!
    """, elem_classes="markdown")

    with gr.Row():
        with gr.Column():
            input_type = gr.Radio(
                ["Excel File", "Researcher Name"], label="Input Type", value="Excel File",
                elem_classes="radio"
            )
            file_input = gr.File(label="Upload Excel File (.xls or .xlsx)", file_types=[".xls", ".xlsx"], visible=True)
            author_input = gr.Textbox(
                label="Researcher Name", visible=False, placeholder="Enter full name, e.g., John Doe"
            )
            api_key_input = gr.Textbox(
                label="Semantic Scholar API Key (optional)", type="password",
                placeholder="Enter your API key"
            )
            current_year = datetime.now().year
            start_year_input = gr.Slider(1900, current_year, value=1900, step=1, label="Start Year")
            end_year_input = gr.Slider(1900, current_year, value=current_year, step=1, label="End Year")
            author_id_input = gr.Textbox(
                label="Selected Author ID", visible=False, placeholder="Enter Author ID (e.g., 123456)"
            )
            submit_button = gr.Button("Submit 🚀", variant="primary")

    with gr.Row():
        with gr.Column():
            authors_df_output = gr.DataFrame(label="Available Authors")
            filtered_df_output = gr.DataFrame(label="Filtered Publications")
            summary_output = gr.DataFrame(label="Year Summary")
        with gr.Column():
            plot1_output = gr.Plot(label="Publication Count")
            plot2_output = gr.Plot(label="Total Citations")

    with gr.Row():
        excel_download = gr.File(label="Download Excel")
        word_download = gr.File(label="Download Word")
        status_output = gr.HTML(label="Status")

    author_list_state = gr.State(value=[])

    def update_inputs(input_type):
        return (
            gr.update(visible=(input_type == "Excel File")),
            gr.update(visible=(input_type == "Researcher Name")),
            gr.update(visible=False)
        )

    input_type.change(
        fn=update_inputs,
        inputs=input_type,
        outputs=[file_input, author_input, author_id_input]
    )

    def process_wrapper(input_type, file, author_name, api_key, start_year, end_year, selected_author_id, author_list):
        excel_file = file.name if file else None
        filtered_df, summary, plot1, plot2, excel_path, word_path, authors_df, status, new_author_list, author_id_visible = process_function(
            input_type, excel_file, author_name, start_year, end_year, api_key, selected_author_id, author_list
        )
        return (
            filtered_df, summary, plot1, plot2, excel_path, word_path, authors_df, status, new_author_list, author_id_visible
        )

    submit_button.click(
        process_wrapper,
        inputs=[input_type, file_input, author_input, api_key_input, start_year_input, end_year_input, author_id_input, author_list_state],
        outputs=[
            filtered_df_output, summary_output, plot1_output, plot2_output, excel_download, word_download,
            authors_df_output, status_output, author_list_state, author_id_input
        ]
    )

if __name__ == "__main__":
    demo.launch()



/tmp/ipython-input-1612452255.py:459: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f7f61a01e13cf2ca1f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
